In [ ]:
!pip install emoji

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import emoji

In [ ]:
current_language='XX' #Set to current language

#File paths: Adjust to relevant paths
file1 = '/en/refugee_OR_refugees_OR_refugees_OR_refugees_OR_Ref - Feb 27, 2024 - 3 27 39 PM.xlsx'
file2 = '/en/refugee_OR_refugees_OR_refugees_OR_refugees_OR_Ref - Feb 27, 2024 - 3 30 39 PM.xlsx'
file3 = '/en/refugee_OR_refugees_OR_refugees_OR_refugees_OR_Ref - Feb 27, 2024 - 3 35 31 PM.xlsx'
file4 = '/en/refugee_OR_refugees_OR_refugees_OR_refugees_OR_Ref - Feb 27, 2024 - 3 39 03 PM.xlsx'
file5 = '/en/refugee_OR_refugees_OR_refugees_OR_refugees_OR_Ref - Feb 27, 2024 - 3 43 56 PM.xlsx'

data = [file1, file2, file3, file4, file5]

df_en = pd.concat((pd.read_excel(path+file) for file in data_english), ignore_index=True)

# 02 Check data

In [ ]:
# Check for duplicate rows and count them
num_duplicates = df.duplicated().sum()
print("Number of duplicate rows in the DataFrame:", num_duplicates)

# Hit sentence duplicates
num_duplicates_column_HitSentence = df['Hit Sentence'].duplicated().sum()
print("Number of duplicates in column 'Hit Sentence':", num_duplicates_column_HitSentence)

# URL duplicates
num_duplicates_column_URL = df['URL'].duplicated().sum()
print("Number of duplicates in column 'URL':", num_duplicates_column_URL)

#Check sources
source_counts = df['Source'].value_counts()
print(source_counts)

# Keep rows where source is 'Twitter'
df = df[df['Source'] == 'Twitter']


In [ ]:
#Check Language
language_counts = df['Language'].value_counts()

print(language_counts)

# Keep rows where language is the current language
df = df[df['Language'] == current_language]

In [ ]:
# Find the maximum length of the 'Hit Sentence'
max_length = df['Hit Sentence'].apply(lambda x: len(str(x))).max()

print("Maximum length of 'Hit Sentence':", max_length)

# Find the minimum length of the 'Hit Sentence'
min_length = df['Hit Sentence'].apply(lambda x: len(str(x))).min()

print("Minimum length of 'Hit Sentence':", min_length)

# 03 EDA

In [ ]:
# Print the date range
date_range = pd.to_datetime(df['Date'])
print("Date range:")
print("From:", date_range.min())
print("To:", date_range.max())

Date range:
From: 2023-01-01 00:11:00
To: 2023-12-31 23:50:00


In [ ]:
# Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Group data by date and count the number of tweets for each date
tweet_counts = df.groupby(df['Date'].dt.date).size()

# Plot the number of tweets over time
plt.figure(figsize=(10, 6))
plt.plot(tweet_counts.index, tweet_counts.values, linestyle='-')
plt.title('Number of Tweets Over Time in English')
plt.xlabel('Date')
plt.ylabel('Number of Tweets')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Smooth the line using a 7-day moving average
smoothed_tweet_counts = tweet_counts.rolling(window=7).mean()

# Plot the smoothed number of tweets over time
plt.figure(figsize=(10, 6))
plt.plot(smoothed_tweet_counts.index, smoothed_tweet_counts.values, linewidth=2)
plt.title('Smoothed Number of Tweets Over Time in English', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Smoothed Number of Tweets', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.yticks(fontsize=10)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
country_counts = df['Country'].value_counts()
print(country_counts)

Identify similar tweets, which are often just Retweets ("RT") or Quote Tweets ("QT")

In [ ]:
# Count tweets that start with "QT"
num_qt_tweets = (df['Hit Sentence'].str.startswith('QT')).sum()
print("Number of tweets starting with 'QT':", num_qt_tweets)

# Count tweets that start with "RT"
num_qt_tweets = (df['Hit Sentence'].str.startswith('RT')).sum()
print("Number of tweets starting with 'RT':", num_qt_tweets)

In [ ]:
sentiment_counts = df['Sentiment'].value_counts()

# Pie plot
plt.figure(figsize=(8, 6))
plt.pie(sentiment_counts, labels=sentiment_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Sentiment Distribution Swedish')
plt.axis('equal')
plt.show()


# 04 Clean data

In [ ]:
# Remove duplicate rows
df = df.drop_duplicates()

# Remove url duplicates
df = df.drop_duplicates(subset=['Hit Sentence'])

# Drop rows with NaN values in 'Hit Sentence' column from the original DataFrame
df = df.dropna(subset=['Hit Sentence'])


In [ ]:
# Drop columns
Echo_columns_to_drop = ['Social Echo Total','Editorial Echo','Twitter Social Echo', 'Facebook Social Echo','Reddit Social Echo']
Engagement_columns_to_drop = ['Views', 'Likes', 'Replies', 'Retweets', 'Comments', 'Shares', 'Reactions', 'Engagement', 'Estimated Views']
Other_columns_to_drop =['Desktop Reach', 'Mobile Reach', 'National Viewership', 'AVE', 'Reach', 'Headline', 'Opening Text', 'Source', 'Document Tags']

Twitter_columns_to_drop = ['Twitter Authority', 'Twitter Followers', 'Twitter Following', 'Twitter Id' , 'Twitter Client',  'Twitter Screen Name', 'Twitter Bio']
User_columns_to_drop = ['Country', 'State','City', 'Subregion', 'Influencer', 'User Profile Url', 'Threads', 'Is Verified']
Tweet_columns_to_drop = ['Tweet Id', 'Parent URL' , 'Key Phrases', 'Document ID']

UNHCR_columns_to_drop = ['Input Name', 'Keywords', 'Alternate Date Format', 'Time']

df_clean = df[:]

df_clean = df.drop(columns=Echo_columns_to_drop)
df_clean = df_clean.drop(columns=Engagement_columns_to_drop)
df_clean = df_clean.drop(columns=Other_columns_to_drop)

df_clean = df_clean.drop(columns=Twitter_columns_to_drop)
df_clean = df_clean.drop(columns=User_columns_to_drop)
df_clean = df_clean.drop(columns=Tweet_columns_to_drop)

df_clean = df_clean.drop(columns=UNHCR_columns_to_drop)

## 04 Text processing - mentions and hashtags

In [ ]:
df_new =  df_clean[:]

# Replace mentions starting with '@' with '@USER'
df_new['Hit Sentence'] = df_new['Hit Sentence'].str.replace(r'@\w+', '@USER')


In [ ]:
# Function to transform emojis into text
def transform_emojis(text):
    return emoji.demojize(text)

# Apply the function to the 'Hit Sentence' column
df_new['Hit Sentence'] = df_new['Hit Sentence'].apply(transform_emojis)


In [ ]:
import re

# Define a regex pattern to match URLs
url_pattern = r'https?://\S+|www\.\S+'

# Function to replace URLs with 'URL'
def replace_urls_with_url(text):
    return re.sub(url_pattern, 'URL', text)

# Apply the function to the 'Hit Sentence' column
df_new['Hit Sentence'] = df_new['Hit Sentence'].apply(replace_urls_with_url)

#Sample tweets for the labelling process

In [ ]:
# Shuffle the rows in df_test
df_shuffled = df_new.sample(frac=1, random_state=42)

# Reset the index of the shuffled DataFrame
df_shuffled.reset_index(drop=True, inplace=True)
df_shuffled.head()

# Sample 600 random rows
df_sampled = df_shuffled.sample(n=600, random_state=42)
df_sampled.info()

In [ ]:
# Group data by date and count the number of tweets for each date
tweet_counts = df_sampled.groupby(df_sampled['Date'].dt.date).size()

# Plot the number of tweets over time
plt.figure(figsize=(10, 6))
plt.plot(tweet_counts.index, tweet_counts.values, linestyle='-')
plt.title('Number of Tweets Over Time in the sample')
plt.xlabel('Date')
plt.ylabel('Number of Tweets')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Save the sampled DataFrame to a CSV file
df_sampled.to_csv('/content/drive/MyDrive/CBS Thesis Lydia & Sara/Data/Data_Labeling/XX', sep='\t', encoding='utf-16', index=False)

Saving the non-sampled tweets to another file:

In [ ]:
df_new = df_new[~df_new['URL'].isin(df_sampled['URL'])]

# Save the DataFrame to a CSV file in Google Drive
df_new.to_csv('/content/drive/MyDrive/CBS Thesis Lydia & Sara/Data/03_Data_Modeling/XX.csv', sep='\t', encoding='utf-16', index=False)